In [1]:
import pandas as pd

df = pd.read_csv("heart.csv")
df.head()

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease
0,40,M,ATA,140,289,0,Normal,172,N,0.0,Up,0
1,49,F,NAP,160,180,0,Normal,156,N,1.0,Flat,1
2,37,M,ATA,130,283,0,ST,98,N,0.0,Up,0
3,48,F,ASY,138,214,0,Normal,108,Y,1.5,Flat,1
4,54,M,NAP,150,195,0,Normal,122,N,0.0,Up,0


In [7]:
from scipy import stats

In [8]:
def remove_outliers_zscore(data, column, lower_threshold=-3, upper_threshold=3):
    z_scores = stats.zscore(data[column])
    return data[(z_scores > lower_threshold) & (z_scores < upper_threshold)]

In [9]:
columns_to_clean = ['RestingBP', 'MaxHR', 'Cholesterol', 'Oldpeak']

df_cleaned = df.copy()
for column in columns_to_clean:
    df_cleaned = remove_outliers_zscore(df_cleaned, column)

In [47]:
df[df.Cholesterol>(df.Cholesterol.mean()+3*df.Cholesterol.std())]

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease
76,32,M,ASY,118,529,0,Normal,130,N,0.0,Flat,1
149,54,M,ASY,130,603,1,Normal,125,Y,1.0,Flat,1
616,67,F,NAP,115,564,0,LVH,160,N,1.6,Flat,0


In [10]:
df_cleaned.shape

(899, 12)

In [12]:
df_cleaned.head()

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease
0,40,M,ATA,140,289,0,Normal,172,N,0.0,Up,0
1,49,F,NAP,160,180,0,Normal,156,N,1.0,Flat,1
2,37,M,ATA,130,283,0,ST,98,N,0.0,Up,0
3,48,F,ASY,138,214,0,Normal,108,Y,1.5,Flat,1
4,54,M,NAP,150,195,0,Normal,122,N,0.0,Up,0


In [17]:
inp = df_cleaned.drop(['HeartDisease'],axis='columns')

In [15]:
out = df_cleaned['HeartDisease']

In [18]:
inp = pd.get_dummies(inp,drop_first=True)

In [19]:
from sklearn.preprocessing import StandardScaler

In [20]:
sc = StandardScaler()

In [21]:
X_scaled = sc.fit_transform(inp)

In [22]:
from sklearn.model_selection import train_test_split

In [34]:
x1,x2,y1,y2= train_test_split(X_scaled,out,test_size=0.2)

In [35]:
from sklearn.svm import SVC

In [36]:
svc = SVC()

In [37]:
svc.fit(x1,y1)

SVC()

In [38]:
svc.score(x2,y2)

0.8888888888888888

In [39]:
from sklearn.ensemble import BaggingClassifier

In [40]:
bag_model = BaggingClassifier(
    estimator=SVC(), 
    n_estimators=100, 
    max_samples=0.8, 
    oob_score=True,
    random_state=0
)

In [45]:
bag_model.fit(x1, y1)
bag_model.oob_score_

0.866481223922114

In [46]:
bag_model.score(x2, y2)

0.8944444444444445